# 使用多模型融合

1 用训练图像分别导出各finetune好了的模型特征，以及对应label

2 构建分类器，并训练，保存权重

3 用test图像导出特征，输入分类器预测

4 输出预测结果到csv

### 1 导出特征和label

In [1]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

import h5py

D:\Users\XX\Anaconda2\envs\keras\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def write_gap(MODEL, image_size, lambda_func=None, weights_file=None, train_imgs_path=None, test_imgs_path=None, model_name=None):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)
    
    base_model = MODEL(input_tensor=x, weights=None, include_top=False)
    base_model.load_weights(weights_file, by_name=True)
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))
    
    
    gen = ImageDataGenerator()
    train_generator = gen.flow_from_directory(train_imgs_path, image_size, shuffle=False, class_mode="categorical",
                                              batch_size=16)
    test_generator = gen.flow_from_directory(test_imgs_path, image_size, shuffle=False,
                                             batch_size=16, class_mode=None)
    
    train_img_nums = train_generator.samples
    test_img_nums = test_generator.samples
    
    train = model.predict_generator(train_generator, (train_img_nums//16) + 1, verbose=1)
    test = model.predict_generator(test_generator, (test_img_nums//16) + 1, verbose=1)

    print("model %s"%(model_name))
    print(train.shape)
    print(test.shape)
    print((train_generator.classes).shape)
    
    train = train[:train_img_nums]
    test = test[:test_img_nums]
    
    print("model %s"%(model_name))
    print(train.shape)
    print(test.shape)
    print((train_generator.classes).shape)
    
    print(train_generator.classes)
    print("#")

    with h5py.File("gap_%s.h5"%(model_name)) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)

# 输入所有的训练样本，后续在分割
#write_gap(ResNet50, (224, 224), 
#          'ResNet50_finetune.h5', '../dataset/train-ready', '../dataset/test')
write_gap(InceptionV3, (299, 299), inception_v3.preprocess_input, 
          'InceptionV3_finetune.h5', '../dataset/train-ready-all', '../dataset/test/', 'InceptionV3')
write_gap(Xception, (299, 299), xception.preprocess_input, 
          'Xception_finetune.h5', '../dataset/train-ready-all', '../dataset/test', 'Xception')

Found 4750 images belonging to 12 classes.
Found 794 images belonging to 1 classes.
50/50 [==============================] - 13s 261ms/step
model InceptionV3
(4750, 2048)
(794, 2048)
(4750,)
model InceptionV3
(4750, 2048)
(794, 2048)
(4750,)
[ 0  0  0 ... 11 11 11]
#
Found 4750 images belonging to 12 classes.
Found 794 images belonging to 1 classes.
50/50 [==============================] - 20s 393ms/step
model Xception
(4750, 2048)
(794, 2048)
(4750,)
model Xception
(4750, 2048)
(794, 2048)
(4750,)
[ 0  0  0 ... 11 11 11]
#


### 2 融合特征

In [3]:
import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(2017)

X_train = []
X_test = []

# 特征是需要融合的，但label是一致的（没有使用乱序），即多个特征融合后对应同一个label

#for filename in ["gap_Xception.h5", "gap_InceptionV3.h5"]:
for filename in ["gap_Xception.h5"]:
    with h5py.File(filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])

print(np.array(X_train).shape)
X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)
print(np.array(X_train).shape)

X_train, y_train = shuffle(X_train, y_train)

print(y_train)
#print("fusion model")
#print("train.shape %d test.shape %d label.shape %d"%(X_train.shape, X_test.shape, y_train.shape))

(1, 4750, 2048)
(4750, 2048)
[5 8 6 ... 1 5 4]


In [4]:
from keras import backend as K

y_train = K.one_hot(y_train, 12)
y_train = K.eval(y_train)
print(y_train[0])
print(y_train[1])
print(y_train[2])

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]


### 3 构建分类器

In [5]:
from keras.models import *
from keras.layers import *

np.random.seed(2017)

input_tensor = Input(X_train.shape[1:])
x = Dropout(0.5)(input_tensor)
predictions = Dense(12, activation='softmax')(x)
model = Model(input_tensor, predictions)

model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2048)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 12)                24588     
Total params: 24,588
Trainable params: 24,588
Non-trainable params: 0
_________________________________________________________________


### 4 训练分类器

In [6]:
batch_size = 128
epochs = 10000

model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=epochs, validation_split=0.1)

D:\Users\XX\Anaconda2\envs\keras\lib\site-packages\ipykernel_launcher.py:4: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  after removing the cwd from sys.path.


Train on 4275 samples, validate on 475 samples
Epoch 1/10000
4275/4275 [==============================] - 1s 335us/step - loss: 4.1718 - acc: 0.1123 - val_loss: 2.5160 - val_acc: 0.1789
Epoch 2/10000
4275/4275 [==============================] - 0s 36us/step - loss: 3.1474 - acc: 0.1357 - val_loss: 2.4616 - val_acc: 0.1832
Epoch 3/10000
4275/4275 [==============================] - 0s 35us/step - loss: 2.7376 - acc: 0.1520 - val_loss: 2.3839 - val_acc: 0.1832
Epoch 4/10000
4275/4275 [==============================] - 0s 34us/step - loss: 2.4673 - acc: 0.1864 - val_loss: 2.3139 - val_acc: 0.1895
Epoch 5/10000
4275/4275 [==============================] - 0s 33us/step - loss: 2.4062 - acc: 0.2002 - val_loss: 2.1924 - val_acc: 0.2274
Epoch 6/10000
4275/4275 [==============================] - 0s 33us/step - loss: 2.3094 - acc: 0.2049 - val_loss: 2.3482 - val_acc: 0.2211
Epoch 7/10000
4275/4275 [==============================] - 0s 35us/step - loss: 2.2747 - acc: 0.2117 - val_loss: 2.2580 - va

4275/4275 [==============================] - 0s 36us/step - loss: 2.0592 - acc: 0.2774 - val_loss: 1.9310 - val_acc: 0.3326
Epoch 60/10000
4275/4275 [==============================] - 0s 35us/step - loss: 2.0752 - acc: 0.2842 - val_loss: 1.9258 - val_acc: 0.3179
Epoch 61/10000
4275/4275 [==============================] - 0s 37us/step - loss: 2.0486 - acc: 0.2849 - val_loss: 2.0170 - val_acc: 0.3074
Epoch 62/10000
4275/4275 [==============================] - 0s 33us/step - loss: 2.0743 - acc: 0.2793 - val_loss: 2.0330 - val_acc: 0.2989
Epoch 63/10000
4275/4275 [==============================] - 0s 35us/step - loss: 2.0583 - acc: 0.2802 - val_loss: 1.9531 - val_acc: 0.3074
Epoch 64/10000
4275/4275 [==============================] - 0s 36us/step - loss: 2.0678 - acc: 0.2791 - val_loss: 2.0964 - val_acc: 0.2274
Epoch 65/10000
4275/4275 [==============================] - 0s 33us/step - loss: 2.0590 - acc: 0.2863 - val_loss: 1.9220 - val_acc: 0.2863
Epoch 66/10000
4275/4275 [================

Epoch 118/10000
4275/4275 [==============================] - 0s 31us/step - loss: 2.0436 - acc: 0.2931 - val_loss: 1.9541 - val_acc: 0.3579
Epoch 119/10000
4275/4275 [==============================] - 0s 31us/step - loss: 2.0338 - acc: 0.2917 - val_loss: 1.9670 - val_acc: 0.2800
Epoch 120/10000
4275/4275 [==============================] - 0s 34us/step - loss: 2.0299 - acc: 0.2940 - val_loss: 1.8954 - val_acc: 0.3053
Epoch 121/10000
4275/4275 [==============================] - 0s 33us/step - loss: 2.0290 - acc: 0.2957 - val_loss: 1.8661 - val_acc: 0.3768
Epoch 122/10000
4275/4275 [==============================] - 0s 35us/step - loss: 2.0440 - acc: 0.2973 - val_loss: 2.0347 - val_acc: 0.2863
Epoch 123/10000
4275/4275 [==============================] - 0s 35us/step - loss: 2.0179 - acc: 0.3050 - val_loss: 2.0096 - val_acc: 0.2421
Epoch 124/10000
4275/4275 [==============================] - 0s 35us/step - loss: 2.0164 - acc: 0.2978 - val_loss: 1.8886 - val_acc: 0.3432
Epoch 125/10000
4275

4275/4275 [==============================] - 0s 35us/step - loss: 2.0017 - acc: 0.3102 - val_loss: 1.8200 - val_acc: 0.3726
Epoch 177/10000
4275/4275 [==============================] - 0s 32us/step - loss: 2.0076 - acc: 0.3069 - val_loss: 1.9898 - val_acc: 0.3074
Epoch 178/10000
4275/4275 [==============================] - 0s 31us/step - loss: 1.9968 - acc: 0.3001 - val_loss: 2.0391 - val_acc: 0.3074
Epoch 179/10000
4275/4275 [==============================] - 0s 34us/step - loss: 2.0160 - acc: 0.3055 - val_loss: 1.9431 - val_acc: 0.2842
Epoch 180/10000
4275/4275 [==============================] - 0s 35us/step - loss: 2.0082 - acc: 0.2989 - val_loss: 1.8204 - val_acc: 0.3347
Epoch 181/10000
4275/4275 [==============================] - 0s 33us/step - loss: 2.0237 - acc: 0.3022 - val_loss: 1.8315 - val_acc: 0.3705
Epoch 182/10000
4275/4275 [==============================] - 0s 32us/step - loss: 2.0244 - acc: 0.3069 - val_loss: 1.9603 - val_acc: 0.3095
Epoch 183/10000
4275/4275 [=========

4275/4275 [==============================] - 0s 33us/step - loss: 2.0071 - acc: 0.3057 - val_loss: 1.9093 - val_acc: 0.3263
Epoch 235/10000
4275/4275 [==============================] - 0s 34us/step - loss: 2.0035 - acc: 0.3043 - val_loss: 1.9441 - val_acc: 0.3516
Epoch 236/10000
4275/4275 [==============================] - 0s 32us/step - loss: 2.0104 - acc: 0.3064 - val_loss: 1.9464 - val_acc: 0.2737
Epoch 237/10000
4275/4275 [==============================] - 0s 31us/step - loss: 2.0302 - acc: 0.2908 - val_loss: 1.8949 - val_acc: 0.3263
Epoch 238/10000
4275/4275 [==============================] - 0s 34us/step - loss: 2.0208 - acc: 0.3013 - val_loss: 1.8949 - val_acc: 0.3453
Epoch 239/10000
4275/4275 [==============================] - 0s 33us/step - loss: 2.0178 - acc: 0.2943 - val_loss: 1.9413 - val_acc: 0.3032
Epoch 240/10000
4275/4275 [==============================] - 0s 39us/step - loss: 2.0069 - acc: 0.2950 - val_loss: 1.9616 - val_acc: 0.2779
Epoch 241/10000
4275/4275 [=========

4275/4275 [==============================] - 0s 34us/step - loss: 2.0110 - acc: 0.3048 - val_loss: 1.8140 - val_acc: 0.3684
Epoch 293/10000
4275/4275 [==============================] - 0s 33us/step - loss: 1.9924 - acc: 0.3067 - val_loss: 1.8312 - val_acc: 0.3874
Epoch 294/10000
4275/4275 [==============================] - 0s 35us/step - loss: 2.0044 - acc: 0.3090 - val_loss: 1.8830 - val_acc: 0.3979
Epoch 295/10000
4275/4275 [==============================] - 0s 36us/step - loss: 2.0021 - acc: 0.3099 - val_loss: 1.9738 - val_acc: 0.2842
Epoch 296/10000
4275/4275 [==============================] - 0s 33us/step - loss: 2.0032 - acc: 0.3004 - val_loss: 1.9331 - val_acc: 0.2800
Epoch 297/10000
4275/4275 [==============================] - 0s 31us/step - loss: 2.0072 - acc: 0.3025 - val_loss: 1.9374 - val_acc: 0.3284
Epoch 298/10000
4275/4275 [==============================] - 0s 32us/step - loss: 2.0183 - acc: 0.3004 - val_loss: 1.9741 - val_acc: 0.2989
Epoch 299/10000
4275/4275 [=========

KeyboardInterrupt: 

### 5 预测测试集

In [ ]:
test_predictions = model.predict(X_test, verbose=1)

### 6 写入CSV

In [ ]:
gen = ImageDataGenerator()
test_generator = gen.flow_from_directory("test2", (224, 224), shuffle=False, 
                                         batch_size=16, class_mode=None)

for i, fname in enumerate(test_generator.filenames):
    index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    df.set_value(index-1, 'label', y_pred[i])